In [1]:
from functions.test_algorithms import *
from functions.random_search import * 
from slim_gsgp_lib.datasets.data_loader import *
import pickle

datasets = [globals()[i] for i in globals() if 'load' in i][2:]

#### Settings

In [2]:
pop_size = 100 
n_iter = 100
n_iter_rs = 50
n_iter_test = 30
p_train = 0.7

#### SLIM 

In [6]:
for dataset_loader in tqdm(datasets[5:]):
    X, y = dataset_loader()
    dataset_name = dataset_loader.__name__.split('load_')[1]

    # Test if the best hyperparameters are valid
    try: 
        with open(f"best_params/best_slim_{dataset_name}_{pop_size}_{n_iter}_True_just_strxo.pkl", 'rb') as f:
            best_params = pickle.load(f)
            print(f"Best hyperparameters for {dataset_name} already exist!")
    except:
        best_params = {}
            
    # Perform random search for both scaled and unscaled versions
    if best_params == {}:
        print(f"Performing random search for {dataset_name}...")
        best_params = random_search_slim(X, y, dataset_name, scale=True, p_train=p_train,
                                            iterations=n_iter_rs, pop_size=pop_size, n_iter=n_iter,
                                            struct_mutation=False, show_progress=False, x_o=True,
                                            gp_xo=False, identifier='just_strxo')
        
        print(f"Random search for {dataset_name} completed!")
    
    # Initialize dictionaries for scaled and unscaled results
    metrics = ['rmse_', 'mape_', 'nrmse_', 'r2_', 'mae_', 'std_rmse_', 'time_stats', 'train_fit', 'test_fit', 'size']
    results = {metric: {} for metric in metrics}

    for algorithm in best_params:
        # Retrieve the best hyperparameters for testing
        args = best_params[algorithm]

        # Test SLIM 
        rm, ma, nrmse, r2, mae, std_rmse, time, train, test, size = test_slim(
            X=X, y=y, args_dict=args, dataset_name=dataset_loader.__name__,
            ms_lower=0, ms_upper=1, n_elites=1,
            iterations=n_iter_test, scale=True, algorithm=algorithm,
            verbose=0, p_train=p_train, show_progress=False,
        )
        
        # Initialize storage for each algorithm if not already present
        for metric in metrics:
            if algorithm not in results[metric]:
                results[metric][algorithm] = []

        # Store results
        results['rmse_'][algorithm].append(rm)
        results['mape_'][algorithm].append(ma)
        results['nrmse_'][algorithm].append(nrmse)
        results['r2_'][algorithm].append(r2)
        results['mae_'][algorithm].append(mae)
        results['std_rmse_'][algorithm].append(std_rmse)
        results['time_stats'][algorithm].append(time)
        results['train_fit'][algorithm].append(train)
        results['test_fit'][algorithm].append(test)
        results['size'][algorithm].append(size)        

        print(f"Results for {algorithm} on {dataset_name} saved!")

    # Save the results to disk
    with open(f"results/SLIM/{dataset_name}_just_strxo.pkl", 'wb') as f:
        pickle.dump(results, f)

    print(f"Results for {dataset_name} saved!")
    print("---------------------------------------------------")

  0%|          | 0/11 [00:00<?, ?it/s]

Best hyperparameters for breast_cancer already exist!
Results for SLIM+SIG2 on breast_cancer saved!
Results for SLIM*SIG2 on breast_cancer saved!
Results for SLIM+ABS on breast_cancer saved!
Results for SLIM*ABS on breast_cancer saved!
Results for SLIM+SIG1 on breast_cancer saved!


  9%|▉         | 1/11 [23:44<3:57:26, 1424.61s/it]

Results for SLIM*SIG1 on breast_cancer saved!
Results for breast_cancer saved!
---------------------------------------------------
Performing random search for concrete_slump...
Random search for concrete_slump completed!
Results for SLIM+SIG2 on concrete_slump saved!
Results for SLIM*SIG2 on concrete_slump saved!
Results for SLIM+ABS on concrete_slump saved!
Results for SLIM*ABS on concrete_slump saved!
Results for SLIM+SIG1 on concrete_slump saved!


 18%|█▊        | 2/11 [1:37:12<7:56:53, 3179.25s/it]

...

Results for SLIM*SIG1 on ld50 saved!
Results for ld50 saved!
---------------------------------------------------
Performing random search for ppb...
Random search for ppb completed!
Results for SLIM+SIG2 on ppb saved!
Results for SLIM*SIG2 on ppb saved!
Results for SLIM+ABS on ppb saved!
Results for SLIM*ABS on ppb saved!
Results for SLIM+SIG1 on ppb saved!


 91%|█████████ | 10/11 [10:01:20<1:05:53, 3953.05s/it]

Results for SLIM*SIG1 on ppb saved!
Results for ppb saved!
---------------------------------------------------
Performing random search for bioav...
Random search for bioav completed!
Results for SLIM+SIG2 on bioav saved!
Results for SLIM*SIG2 on bioav saved!
Results for SLIM+ABS on bioav saved!
Results for SLIM*ABS on bioav saved!
Results for SLIM+SIG1 on bioav saved!


100%|██████████| 11/11 [11:24:06<00:00, 3731.52s/it]  

Results for SLIM*SIG1 on bioav saved!
Results for bioav saved!
---------------------------------------------------
